In [44]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim

In [45]:
class FullyConnectedNN(nn.Module):
    def __init__(self, input_size=4096, hidden_size=256, output_size=1):
        super(FullyConnectedNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.tanh = nn.Tanh()
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        return out

In [46]:
# Подготовка данных (пример)
X = pd.read_csv(r'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\X_train_32_128.csv')
y = pd.read_csv(r'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\y_train_32_128.csv')

In [23]:
X.iloc[0]

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
4091    0.0
4092    0.0
4093    0.0
4094    0.0
4095    1.0
Name: 0, Length: 4096, dtype: float64

In [14]:
y.head(5)

,value
0,-0.013916
1,-0.008545
2,-0.007690
3,-0.003662
4,-0.003662


In [47]:
Xi = torch.tensor(X.values, dtype=torch.float32).cuda() 
yi = torch.tensor(y.values, dtype=torch.float32).cuda() 

In [48]:
X_tensor = Xi
y_tensor = yi

In [37]:
len(Xi[0])

4096

In [38]:
yi[1]

tensor([-0.0085], device='cuda:0')

In [49]:
model = FullyConnectedNN().cuda()

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()  # Функция потерь для регрессии
optimizer = optim.Adam(model.parameters(), lr=0.000001)  # Оптимизатор Adam с learning rate 0.001

# Обучение модели
num_epochs = 4
for epoch in range(num_epochs):
    running_loss = 0.0
    for i in range(len(X_tensor)):
        inputs = X_tensor[i]
        labels = torch.tensor([y_tensor[i].item()], dtype=torch.float32).cuda()

        # Обнуление градиентов параметров
        optimizer.zero_grad()

        # Прямой проход
        outputs = model(inputs)

        # Рассчет функции потерь
        loss = criterion(outputs, labels)

        # Обратный проход и оптимизация
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(X_tensor)}")

print("Обучение завершено")

Epoch [1/4], Loss: 0.005477436726187487
Epoch [2/4], Loss: 0.0028857010235578146
Epoch [3/4], Loss: 0.0019613521202916633
Epoch [4/4], Loss: 0.0014447545290165976
Обучение завершено


In [50]:
model.forward(X_tensor[400])

tensor([0.0086], device='cuda:0', grad_fn=<TanhBackward0>)

In [51]:
model.forward(X_tensor[3])

tensor([-0.0263], device='cuda:0', grad_fn=<TanhBackward0>)

In [52]:
torch.save(model.state_dict(), 'C:\PycharmProjects\ETS_Autopilot\dataset_for_wheel_nn\weight_32_128.pth')